 ## Assignment 2

 This assignment is aimed to give you some practice using Jupyter Notebooks, R, and interpretting statistical output using real-world data. The notebook below will be used to generate the statistical output and you will also write up answers to specific questions related to the statistical output. The assignment will be guided and much of the R code will be provided for you, but specific aspects of the R code you will be asked to interact with and ultimately make a decision about appropriate values to include. The notebook should be run from the first code cell in sequential order, this means that you must run the beginning cells in order to be able to have access to the R packages needed for the assignment and that the data are read in appropriately.

 You may work in groups of up to 3 to complete the assignment. In these situations, please turn in one assignment in ICON with all group members names on the submission.

 *Assignment 2 Due*: **Sunday, March 21, by 11:59 pm**

 ## Description of the Data

 These data are weather data from Australia weather stations between 2007 and 2017.

 + **Date**: The date of weather observation.
 + **Location**: Location of weather observation.
 + **MinTemp**: Minimum temperature.
 + **MaxTemp**: Maximum temperature.
 + **Rainfall**: Daily rainfall (in mm).
 + **WindGustDir**: Wind gust direction
 + **WindGustSpeed**: Wind gust speed (in km/h)
 + **WindGustDir9am**: Wind gust direction at 9am.
 + **WindGustDir3pm**: Wind gust direction at 3pm.
 + **WindGustSpeed9am**: Wind gust speed (in km/h) at 9am
 + **WindGustSpeed3pm**: Wind gust speed (in km/h) at 3pm
 + **Humidity9am**: Humidity (percent) at 9am
 + **Humidity3pm**: Humidity (percent) at 3pm
 + **Pressure9am**: Atmospheric pressure (hpa) at 9am
 + **Pressure3pm**: Atmospheric pressure (hpa) at 3pm
 + **Temp9am**: Temperature (in Celsius) at 9am
 + **Temp3pm**: Temperature (in Celsius) at 3 pm
 + **RainToday**: Boolean: 1 if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0
 + **RainTomorrow**: The target variable. Did it rain tomorrow? Yes/No
 + **year**: The year of observation
 + **month**: The month of observation, represented as text labels
 + **day**: The day of the observation within each month
 + **day_labels**: The day of the week, (e.g. Sun = Sunday)

 Please don't hesitate to reach out with any data questions about the structure and interpretation of the variables in the data.

 ## Assignment Setup

 **Run this cell first upon opening the notebook everytime** This cell loads the R packages and prepares the data for you.

In [ ]:
library(tidyverse)
library(ggformula)
library(mosaic)
library(rpart)
library(rpart.plot)
library(rsample)
library(lubridate)

theme_set(theme_bw(base_size = 14))

aus_weather <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/master/data-raw/weatherAUS.csv",
                        guess_max = 100000) %>%
                        select(-Evaporation, -Sunshine, -Cloud9am, -Cloud3pm, -RISK_MM) %>%
                        na.omit() %>%
                        mutate(year = year(Date),
                               month = month(Date, label = TRUE),
                               day = day(Date),
                               day_labels = wday(Date, label = TRUE),
                               RainTomorrow_num = ifelse(RainTomorrow == 'Yes', 1, 0))

head(aus_weather)

 ## Question 1
 Explore the `MaxTemp` distribution visually using the code provided below.

 Complete the code by filling in the appropriate variable where the "^^" are and fill in the visualization type you are most comfortable with where "??" are located. Finally, replace the "$$" with an appropriate plot title and x-axis label that are descriptive. **1 pt**

In [ ]:
gf_??(~ ^^, data = aus_weather) %>%
  gf_labs(title = "$$", 
          x = "$$") 


 ## Question 2
 Based on the figure created in question 2 above, estimate the mean and median from the figure. Provide a sentence rationale for why you think the mean and median are approximately what you estimated. Be as specific as you can about features in the figure that you considered when picking values for the mean and median. Note, no need to be exact or compute the statistics yet (this will come later), rather provide your best guess solely from the figure create in question 2. **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

 ## Question 3
 Use the following code to compute descriptive statistics that summarize the center and variability for the `MaxTemp` variable.

 Complete the code by filling in the appropriate descriptive functions where the "&&" are located in the code below. Functions that may be useful here could include: `mean`, `median`, `sd`, `var`, `IQR`, `min`, `max`, `length`. **1 pt**

In [ ]:
aus_weather %>%
  df_stats(~ MaxTemp, &&)


 ## Question 4a
 For each of the statistics related to the **center of the distribution** calculated in question 3 above, provide a one or two sentence discussion of what the measures of **center** tell about the `MaxTemp` attribute. More specifically, interpret the statistics in the context of this particular data. **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

 ## Question 4b
 For each of the statistics related to the **variation of the distribution** calculated in question3 above, provide a one or two sentence discussion of what the measures of **variation** tell about the `MaxTemp` attribute. More specifically, interpret the statistics in the context of this particular data. **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

 ## Question 4c
 Using the statistics calculated in question 3 above, provide a one or two sentence discussion of what the statistics tell about the **shape** of the `MaxTemp` attribute. More specifically, what do the statistics suggest about the shape of the distribution? **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

 ## Question 5
 Hypothesize which variables may help to explain variation in the `MaxTemp` variable. Put another way, what are some important variables that may account for differences in the maximum temperature for that day. These may include variables that are in the data or other variables that are not in the data, think big picture here about other things that may the home sale price. **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

 ## Question 6
 Compute conditional multivariate descriptive statistics for the `MaxTemp` variable based on a variable you think may help to explain differences in the maximum temperature for that day. This may include variables you discussed in question 5 above.

 Complete the code by filling in the appropriate outcome variable in place of "^^", the variable(s) of interest in place of "$$", and the descriptive functions where the "&&" are located in the code below. Functions that may be useful here could include: `mean`, `median`, `sd`, `var`, `IQR`, `min`, `max`, `length`. **1 pt**

In [ ]:
aus_weather %>%
  df_stats(^^ ~ $$, &&)


 ## Question 7
 Based on the descriptive statistics computed in question 6 above, interpret important differences in the descriptive statistics by the attribute(s) that you explored in question 6. Be as specific as possible about the differences and interpret the differences in the context of the data. **2 pts**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

 ## Question 8
  Before predicting weather it is going to rain tomorrow based on the weather observations from the previous day, let's first explore how often it rains tomorrow.
  Complete the code below to generate descriptive statistics on the numeric `RainTomorrow_num` attribute by filling in the descriptive functions where the "&&" are located in the code below. Functions that may be useful here could include: `mean`, `median`, `sd`, `var`, `IQR`, `min`, `max`, `length`. **1 pt**

In [ ]:
aus_weather %>%
  df_stats(~ RainTomorrow_num, &&)


  ## Question 9
  Discuss how often it rains compared to not raining, citing specific evidence from question 8. How will the number of times it rains tomorrow impact the classification model that will be fitted next? Be as specific in your discussion of the impacts as possible. **1 pt**.

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

  ## Question 10
  We are now going to explore which data attributes (i.e. weather observations) that are most important in predicting if it is going to rain tomorrow based on the weather observations from the previous day. To see which attributes are important, we will fit a classification tree to these data using the `rpart()` function.
  
  Complete the code within the `rpart()` function below by replacing "$$" with variables included in the data that you think would be important in days in which it will rain tomorrow. Note, separate variables with a `+` symbol and I recommend not using the location attribute. **1 pt**

In [ ]:
set.seed(2020)
aus_weather_split <- initial_split(aus_weather, prop = .7)
aus_weather_train <- training(aus_weather_split)
aus_weather_test <- testing(aus_weather_split)


# Fit classification tree
aus_weather_class <- rpart(RainTomorrow ~ $$, 
                    method = 'class', data = aus_weather_train)

rpart.plot(aus_weather_class, roundint = FALSE, type = 3, branch = .3)


  ## Question 11
  Based on the classification tree figure created from question 10 above, what are the most important variables that help to predict whether it will rain tomorrow? Which variable was the most important? **1 pt**

  *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

  ## Question 12
  The code below will use the model fitted above and produce predicted values for the withheld test data. 

In [ ]:
aus_weather_test <- aus_weather_test %>%
  mutate(rain_tomorrow_predict = predict(aus_weather_class, 
                                    newdata = aus_weather_test, 
                                    type = 'class'))

aus_weather_test %>%
  mutate(same_class = ifelse(RainTomorrow == rain_tomorrow_predict, 1, 0)) %>%
  df_stats(~ same_class, mean, sum, length)


 Do you feel the classification tree did a good job at accurately predicting whether it will rain tomorrow? Be as specific as possible citing the descriptive statistics computed above. **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*

  ## Question 13
  The code below will create a bar chart is created that shows the accuracy of the predictions compared to the actual observed data.

In [ ]:
gf_bar(~ RainTomorrow, fill = ~rain_tomorrow_predict, data = aus_weather_test, 
       position = 'fill') %>% 
       gf_labs(y = "Proportion", 
       x = "Did it really rain tomorrow?", 
       fill = "Model Predicted Rainfall")


 Do you feel the classification tree still did a good job at accurately predicting whether it will rain tomorrow when looking at the bar chart compared to only the descriptive statistics? Be as specific as possible citing the bar chart created above. **1 pt**

 *Write your response in this cell by double clicking on this text. When finished typing your response, hit control + enter to convert the text.*